<a href="https://colab.research.google.com/github/milagrosonofri/Problem_Set_1/blob/main/Copia_de_sepa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
import glob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
CARPETA_DATOS = "/content/drive/MyDrive/SEPA"

## Bajar archivos

In [ ]:
import requests
from bs4 import BeautifulSoup
import os
from urllib.parse import urljoin

def download_zip_files(url, download_folder, extension="zip"):
    # Create download folder if it doesn't exist
    os.makedirs(download_folder, exist_ok=True)

    # Send a GET request to the URL
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to access {url}")
        return

    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all links ending with .zip
    zip_links = [urljoin(url, a['href']) for a in soup.find_all('a', href=True) if a['href'].endswith(extension)]

    if not zip_links:
        print("No ZIP files found.")
        return

    # Download each ZIP file
    for link in zip_links:
        file_name = os.path.join(download_folder, os.path.basename(link))
        print(f"Downloading {link} to {file_name}...")

        # Stream download the file
        with requests.get(link, stream=True) as r:
            if r.status_code == 200:
                with open(file_name, 'wb') as f:
                    for chunk in r.iter_content(chunk_size=8192):
                        f.write(chunk)
                print(f"Downloaded: {file_name}")
            else:
                print(f"Failed to download: {link}")


### Para bajarlos de GitHub

https://github.com/catdevnull/sepa-precios-metadata/blob/main/index.md

In [ ]:
download_zip_files("https://github.com/catdevnull/sepa-precios-metadata/blob/main/index.md", download_folder=f"{CARPETA_DATOS}/downloaded_from_github", extension="tar.zst")

Downloaded: /content/drive/MyDrive/SEPA/downloaded_from_github/4cf559c6-8665-40c1-8149-4eb3ccf40622-revID-5de12e2b-3b53-4cb9-aa47-9442a9e17f83-sepa_lunes.zip-repackaged.tar.zst
Downloaded: /content/drive/MyDrive/SEPA/downloaded_from_github/fa18da68-65f4-427a-9167-528e9bcb9ec9-revID-a5e5c6a6-ba57-4884-b844-83e58d278e96-sepa_martes.zip-repackaged.tar.zst
Downloaded: /content/drive/MyDrive/SEPA/downloaded_from_github/4d35b6b1-dc34-47d3-8a26-76dc45d2fb88-revID-b32c77b9-7154-43b9-b483-eb930341dc03-precios_21.08.2024.zip-repackaged.tar.zst
Downloaded: /content/drive/MyDrive/SEPA/downloaded_from_github/d076720f-a7f0-4af8-b1d6-1b99d5a90c14-revID-7e11a3d4-77b5-4f61-a1d6-442c2e864d34-sepa_jueves.zip-repackaged.tar.zst
Downloaded: /content/drive/MyDrive/SEPA/downloaded_from_github/91bc072a-4726-44a1-85ec-4a8467aad27e-revID-fabc7cfe-ddfc-4c76-adb5-8455604e549f-sepa_viernes.zip-repackaged.tar.zst
Downloaded: /content/drive/MyDrive/SEPA/downloaded_from_github/b3c3da5d-213d-41e7-8d74-f23fda0a3c30-rev

KeyboardInterrupt: 

In [ ]:
# PARA DESCOMPRIMIR TAR.ZST (NO LO PROBAMOS)


!pip install zstandard

import tarfile
import zstandard as zstd
import os

def extract_tar_zst(file_path, extract_to):
    # Create the output directory if it doesn't exist
    os.makedirs(extract_to, exist_ok=True)

    # Open the .zst file and decompress it
    with open(file_path, 'rb') as compressed_file:
        dctx = zstd.ZstdDecompressor()
        # Create a decompressed file object
        with dctx.stream_reader(compressed_file) as decompressed_file:
            # Open the decompressed tar archive
            with tarfile.open(fileobj=decompressed_file, mode='r:') as tar:
                tar.extractall(path=extract_to)
                print(f"Extracted all files to {extract_to}")

# Example usage
file_path = f'{CARPETA_DATOS}/downloaded_from_github/0a9069a9-06e8-4f98-874d-da5578693290-revID-1f702d76-6459-4c30-8f3c-5714fe848bde-sepa_lunes.zip-repackaged.tar.zst'
extract_to = f'{CARPETA_DATOS}/downloaded_from_github/0a9069a9-06e8-4f98-874d-da5578693290-revID-1f702d76-6459-4c30-8f3c-5714fe848bde-sepa_lunes'
extract_tar_zst(file_path, extract_to)

  Using cached zstandard-0.23.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.0 kB)
Using cached zstandard-0.23.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.4 MB)
Extracted all files to /content/drive/MyDrive/SEPA/downloaded_from_github/0a9069a9-06e8-4f98-874d-da5578693290-revID-1f702d76-6459-4c30-8f3c-5714fe848bde-sepa_lunes


In [ ]:
def procesar_datos_comercio(carpeta_comercio):

    comercio_df = pd.read_csv(f"{carpeta_comercio}/comercio.csv", sep="|")
    fecha = comercio_df.iloc[-1,:]

    comercio_df = comercio_df.iloc[:-1,:]

    sucursales_df = pd.read_csv(f"{carpeta_comercio}/sucursales.csv", sep="|")
    sucursales_df = sucursales_df.iloc[:-1,:]

    productos_df = pd.read_csv(f"{carpeta_comercio}/productos.csv", sep="|")
    productos_df = productos_df.iloc[:-1,:]

    return comercio_df, sucursales_df, productos_df, fecha

In [ ]:
#!pip install tqdm
from tqdm import tqdm # BARRA DE PROGRESO

In [ ]:
!pip install tqdm

In [ ]:
diccionario = {}

for subcarpeta in tqdm(glob.glob(f"{extract_to}/*")):

    # print(archivo_zip)
    # carpeta = archivo_zip.replace(".zip-repackaged.tar.zst", "")
    # unzip_file(archivo_zip, carpeta)

    try:
        diccionario[subcarpeta] = procesar_datos_comercio(subcarpeta)
    except Exception as e:
        print(f"La carpeta {subcarpeta} no se pudo procesar.")
        print(e)

  0%|          | 0/30 [00:00<?, ?it/s]<ipython-input-11-f70ba8ddc542>:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  productos_df = pd.read_csv(f"{carpeta_comercio}/productos.csv", sep="|")
  3%|▎         | 1/30 [00:00<00:13,  2.20it/s]<ipython-input-11-f70ba8ddc542>:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  productos_df = pd.read_csv(f"{carpeta_comercio}/productos.csv", sep="|")
  7%|▋         | 2/30 [00:06<01:52,  4.00s/it]<ipython-input-11-f70ba8ddc542>:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  productos_df = pd.read_csv(f"{carpeta_comercio}/productos.csv", sep="|")
 13%|█▎        | 4/30 [00:07<00:42,  1.62s/it]<ipython-input-11-f70ba8ddc542>:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  productos_df = pd.read_csv(f"{carpeta_comercio}/produ

La carpeta /content/drive/MyDrive/SEPA/downloaded_from_github/0a9069a9-06e8-4f98-874d-da5578693290-revID-1f702d76-6459-4c30-8f3c-5714fe848bde-sepa_lunes/sepa_1_comercio-sepa-16_2024-08-26_09-05-12 no se pudo procesar.
Error tokenizing data. C error: Expected 17 fields in line 27022, saw 18



<ipython-input-11-f70ba8ddc542>:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  productos_df = pd.read_csv(f"{carpeta_comercio}/productos.csv", sep="|")
 53%|█████▎    | 16/30 [00:12<00:04,  2.84it/s]<ipython-input-11-f70ba8ddc542>:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  productos_df = pd.read_csv(f"{carpeta_comercio}/productos.csv", sep="|")
 60%|██████    | 18/30 [00:17<00:11,  1.02it/s]<ipython-input-11-f70ba8ddc542>:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  productos_df = pd.read_csv(f"{carpeta_comercio}/productos.csv", sep="|")
 70%|███████   | 21/30 [00:17<00:05,  1.56it/s]<ipython-input-11-f70ba8ddc542>:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  productos_df = pd.read_csv(f"{carpeta_comercio}/productos.csv", sep="|")
 73%|███████▎  

La carpeta /content/drive/MyDrive/SEPA/downloaded_from_github/0a9069a9-06e8-4f98-874d-da5578693290-revID-1f702d76-6459-4c30-8f3c-5714fe848bde-sepa_lunes/dataset-info.json no se pudo procesar.
[Errno 20] Not a directory: '/content/drive/MyDrive/SEPA/downloaded_from_github/0a9069a9-06e8-4f98-874d-da5578693290-revID-1f702d76-6459-4c30-8f3c-5714fe848bde-sepa_lunes/dataset-info.json/comercio.csv'


In [ ]:
df = diccionario['/content/drive/MyDrive/SEPA/downloaded_from_github/0a9069a9-06e8-4f98-874d-da5578693290-revID-1f702d76-6459-4c30-8f3c-5714fe848bde-sepa_lunes/sepa_2_comercio-sepa-10_2024-08-26_01-05-11'][2]

In [ ]:
# df.id_producto.isna().sum() # .nunique()
# df.productos_descripcion.isna().sum()

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact

@interact
def mostrar_productos(comercio=widgets.SelectionSlider(options=diccionario.keys())):
    return diccionario[comercio][2]

interactive(children=(SelectionSlider(description='comercio', options=('/content/drive/MyDrive/SEPA/downloaded…

In [ ]:
key1 = list(diccionario.keys())[0]

In [ ]:
import datetime

In [ ]:
key1

'/content/drive/MyDrive/SEPA/downloaded_from_github/0a9069a9-06e8-4f98-874d-da5578693290-revID-1f702d76-6459-4c30-8f3c-5714fe848bde-sepa_lunes/sepa_1_comercio-sepa-2002_2024-08-26_09-05-12'

In [ ]:
fecha = diccionario[key1][3][0].split(" ")[-1].split("T")[0]
fecha = datetime.datetime.strptime(fecha, "%Y-%m-%d")

<ipython-input-19-d1b5cca1deff>:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  fecha = diccionario[key1][3][0].split(" ")[-1].split("T")[0]


Tablas:
- Precios (principal): comercio_id, bandera_id, sucursal_id, producto, fecha, precio.
- Comercio: comercio_id, bandera_id, nombre
- Sucursal: comercio_id, bandera_id, sucursal_id, nombre, direccion, latitud, longitud
- Producto: producto_id, descripcion, productos_cantidad_presentacion, productos_unidad_medida_presentacion, productos_marca

In [ ]:
import sqlite3
conn = sqlite3.connect(f'{CARPETA_DATOS}/todos_los_precios.sqlite')
cursor = conn.cursor()

In [ ]:
columnas_precios = ["id_comercio", "id_bandera", "id_sucursal", "id_producto", "productos_precio_lista"]
precios_df = diccionario[key1][2][columnas_precios]
precios_df = precios_df.assign(fecha=fecha)
precios_df

,id_comercio,id_bandera,id_sucursal,id_producto,productos_precio_lista,fecha
0,2002,1.0,128.0,7.798305e+12,854999.0,2024-08-26
1,2002,1.0,110.0,7.798305e+12,854999.0,2024-08-26
2,2002,1.0,113.0,7.798305e+12,854999.0,2024-08-26
3,2002,1.0,114.0,7.798305e+12,854999.0,2024-08-26
4,2002,1.0,116.0,7.798305e+12,854999.0,2024-08-26
...,...,...,...,...,...,...
53312,2002,1.0,99.0,7.796885e+12,152999.0,2024-08-26
53313,2002,1.0,150.0,7.796885e+12,152999.0,2024-08-26
53314,2002,1.0,170.0,7.796885e+12,152999.0,2024-08-26
53315,2002,1.0,199.0,7.796885e+12,152999.0,2024-08-26


In [ ]:
precios_df.id_comercio = precios_df.id_comercio.astype(int)
precios_df.id_bandera = precios_df.id_bandera.astype(int)
precios_df.id_sucursal = precios_df.id_sucursal.astype(int)
precios_df.id_producto = precios_df.id_producto.astype(int)
precios_df

,id_comercio,id_bandera,id_sucursal,id_producto,productos_precio_lista,fecha
0,2002,1,128,7798304570499,854999.0,2024-08-26
1,2002,1,110,7798304570499,854999.0,2024-08-26
2,2002,1,113,7798304570499,854999.0,2024-08-26
3,2002,1,114,7798304570499,854999.0,2024-08-26
4,2002,1,116,7798304570499,854999.0,2024-08-26
...,...,...,...,...,...,...
53312,2002,1,99,7796885483276,152999.0,2024-08-26
53313,2002,1,150,7796885483276,152999.0,2024-08-26
53314,2002,1,170,7796885483276,152999.0,2024-08-26
53315,2002,1,199,7796885483276,152999.0,2024-08-26


Tablas de comercio, sucursal y producto



In [ ]:
columnas_comercio = ["id_comercio", "id_bandera", "comercio_bandera_nombre"]
columnas_sucursal = ["id_comercio", "id_bandera", "id_sucursal", "sucursales_tipo", "sucursales_localidad", "sucursales_latitud", "sucursales_longitud"]
columnas_producto = ["id_producto", "productos_descripcion", "productos_cantidad_presentacion",
                     "productos_unidad_medida_presentacion", "productos_marca"]


In [ ]:
# Proceso Comercio
comercio_df = diccionario[key1][0][columnas_comercio]

# Proceso Sucursal
sucursal_df = diccionario[key1][1][columnas_sucursal]

# Proceso Producto
producto_df = diccionario[key1][2][columnas_producto]


In [ ]:
precios_df.to_sql('precios', conn, if_exists='replace', index=False)
#conn.commit()
#conn.close()

53317

In [ ]:

precios_df.to_sql('precios', conn, if_exists='append', index=False)
comercio_df.to_sql('comercio', conn, if_exists='append', index=False)
sucursal_df.to_sql('sucursal', conn, if_exists='append', index=False)
producto_df.to_sql('productos', conn, if_exists='append', index=False)

conn.commit()  # Guardo los cambios en la base de datos
#conn.close()


In [ ]:
# estructura de la tabla 'precios'
cursor.execute("PRAGMA table_info(precios);")
print(cursor.fetchall())


[(0, 'id_comercio', 'INTEGER', 0, None, 0), (1, 'id_bandera', 'INTEGER', 0, None, 0), (2, 'id_sucursal', 'INTEGER', 0, None, 0), (3, 'id_producto', 'INTEGER', 0, None, 0), (4, 'productos_precio_lista', 'REAL', 0, None, 0), (5, 'fecha', 'TIMESTAMP', 0, None, 0)]


In [ ]:
# estructura de la tabla 'comercio'
cursor.execute("PRAGMA table_info(comercio);")
print(cursor.fetchall())

[(0, 'id_comercio', 'TEXT', 0, None, 0), (1, 'id_bandera', 'REAL', 0, None, 0), (2, 'comercio_bandera_nombre', 'TEXT', 0, None, 0)]


In [ ]:
# estructura de la tabla 'sucursal'
cursor.execute("PRAGMA table_info(sucursal);")
print(cursor.fetchall())

[(0, 'id_comercio', 'TEXT', 0, None, 0), (1, 'id_bandera', 'REAL', 0, None, 0), (2, 'id_sucursal', 'REAL', 0, None, 0), (3, 'sucursales_tipo', 'TEXT', 0, None, 0), (4, 'sucursales_localidad', 'TEXT', 0, None, 0), (5, 'sucursales_latitud', 'REAL', 0, None, 0), (6, 'sucursales_longitud', 'REAL', 0, None, 0)]


In [ ]:
# estructura de la tabla 'productos'
cursor.execute("PRAGMA table_info(productos);")
print(cursor.fetchall())

[(0, 'id_producto', 'REAL', 0, None, 0), (1, 'productos_descripcion', 'TEXT', 0, None, 0), (2, 'productos_cantidad_presentacion', 'REAL', 0, None, 0), (3, 'productos_unidad_medida_presentacion', 'TEXT', 0, None, 0), (4, 'productos_marca', 'TEXT', 0, None, 0)]


In [ ]:
query = '''
SELECT p.id_comercio, p.id_sucursal, p.id_producto, p.id_bandera, pr.productos_descripcion,
       p.productos_precio_lista, c.comercio_bandera_nombre
FROM precios p
JOIN producto pr ON p.id_producto = pr.id_producto
JOIN comercio c ON p.id_comercio = c.id_comercio AND p.id_bandera = c.id_bandera
ORDER BY productos_precio_lista DESC LIMIT 10
'''

In [ ]:
result = pd.read_sql(query, conn)
result


,id_comercio,id_sucursal,id_producto,id_bandera,productos_descripcion,productos_precio_lista,comercio_bandera_nombre


In [ ]:
cursor.execute('''
    CREATE INDEX IF NOT EXISTS idx_id_producto ON precios (id_producto)
''')

cursor.execute('''
    CREATE INDEX IF NOT EXISTS idx_id_producto ON producto (id_producto)
''')

conn.commit()

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="joeddav/xlm-roberta-large-xnli")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/734 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [ ]:
producto_df.productos_descripcion.iloc[:200].to_list()

['COMBO FLORENCIA ANAFE 6748I HORNO ELECT.',
 'COMBO FLORENCIA ANAFE 6748I HORNO ELECT.',
 'COMBO FLORENCIA ANAFE 6748I HORNO ELECT.',
 'COMBO FLORENCIA ANAFE 6748I HORNO ELECT.',
 'COMBO FLORENCIA ANAFE 6748I HORNO ELECT.',
 'COMBO FLORENCIA ANAFE 6748I HORNO ELECT.',
 'COMBO FLORENCIA ANAFE 6748I HORNO ELECT.',
 'COMBO FLORENCIA ANAFE 6748I HORNO ELECT.',
 'COMBO FLORENCIA ANAFE 6748I HORNO ELECT.',
 'COMBO FLORENCIA ANAFE 6748I HORNO ELECT.',
 'COMBO FLORENCIA ANAFE 6748I HORNO ELECT.',
 'COMBO FLORENCIA ANAFE 6748I HORNO ELECT.',
 'COMBO FLORENCIA ANAFE 6748I HORNO ELECT.',
 'COMBO FLORENCIA ANAFE 6748I HORNO ELECT.',
 'COMBO FLORENCIA ANAFE 6748I HORNO ELECT.',
 'COMBO FLORENCIA ANAFE 6748I HORNO ELECT.',
 'COMBO FLORENCIA ANAFE 6748I HORNO ELECT.',
 'COMBO FLORENCIA ANAFE 6748I HORNO ELECT.',
 'COMBO FLORENCIA ANAFE 6748I HORNO ELECT.',
 'COMBO FLORENCIA ANAFE 6748I HORNO ELECT.',
 'COMBO FLORENCIA ANAFE 6748I HORNO ELECT.',
 'COMBO FLORENCIA ANAFE 6748I HORNO ELECT.',
 'COMBO FL

In [ ]:
result = classifier(
    "cepillo de dientes",
    candidate_labels=["comida", "bebida", "cuidado personal", "limpieza del hogar", "tecnología", "electrónica"],
    hypothesis_template="Este producto pertenece a la categoría de {}"
)
print(result)

{'sequence': 'cepillo de dientes', 'labels': ['cuidado personal', 'limpieza del hogar', 'tecnología', 'comida', 'electrónica', 'bebida'], 'scores': [0.8441327810287476, 0.10267408937215805, 0.020099066197872162, 0.014154705218970776, 0.01336676999926567, 0.005572659894824028]}
